In [3]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
#             bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
#             pt.rewrite.RM3(indexref) >> 
#             pt.BatchRetrieve(indexref, wmodel="BM25"))
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels, 
                                names=['tuned_bm25_rm3','orignal_bm25_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20original_tuned_bm25_rm3', '20orignal_bm25_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21original_tuned_bm25_rm3', '21orignal_bm25_rm3'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/original_tuned_bm25_rm3.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.584666
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c7af90 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18334b98 at 0x7f9f19e385b0>>) fb_terms=12', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c7af90 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18334b98 at 0x7f9f19e385b0>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c7af90 jclass=org/terrier/querying/RM3 jself=<LocalRe

/tmp/ipykernel_1615848/489219154.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels,


\begin{tabular}{llrrr}
\toprule
{} &              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    tuned\_bm25\_rm3 &    0.457078 &     0.411862 &  0.679850 \\
1 &  orignal\_bm25\_rm3 &    0.433057 &     0.384884 &  0.634694 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/489219154.py:39: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20original_tuned_bm25_rm3', '20orignal_bm25_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                       name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20original\_tuned\_bm25\_rm3 &    0.355583 &     0.352399 &  0.708930 \\
1 &         20orignal\_bm25\_rm3 &    0.308863 &     0.307938 &  0.643398 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/489219154.py:42: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21original_tuned_bm25_rm3', '21orignal_bm25_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                       name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21original\_tuned\_bm25\_rm3 &    0.556543 &     0.470135 &  0.651351 \\
1 &         21orignal\_bm25\_rm3 &    0.554767 &     0.460291 &  0.626163 \\
\bottomrule
\end{tabular}



In [5]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels, names=[folders[i] + '_tuned_bm_rm3', folders[i] + '_orignal_bm_rm3'],
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_bm_rm3', '20' + folders[i] + '_bm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_bm_rm3', '21'+ folders[i] + '_bm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_bm_rm3.csv")
            i+=1


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.558365
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9fa8d2b1d0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18ff8970 at 0x7f9f19d91ed0>>) fb_terms=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9fa8d2b1d0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18ff8970 at 0x7f9f19d91ed0>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9fa8d2b1d0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x5

/tmp/ipykernel_1615848/2031853054.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels, names=[folders[i] + '_tuned_bm_rm3', folders[i] + '_orignal_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    conclusions\_tuned\_bm\_rm3 &    0.457201 &     0.413853 &  0.757675 \\
1 &  conclusions\_orignal\_bm\_rm3 &    0.394756 &     0.375585 &  0.710391 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/2031853054.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_bm_rm3', '20' + folders[i] + '_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                       name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20conclusionstuned\_bm\_rm3 &    0.381014 &     0.358802 &  0.770298 \\
1 &       20conclusions\_bm\_rm3 &    0.305337 &     0.304660 &  0.695431 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    pools\_conclusions\_tuned\_bm\_rm3 &    0.432140 &     0.407370 &  0.755836 \\
1 &  pools\_conclusions\_orignal\_bm\_rm3 &    0.396597 &     0.372488 &  0.708422 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/2031853054.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_bm_rm3', '20' + folders[i] + '_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20pools\_conclusionstuned\_bm\_rm3 &    0.376738 &     0.356485 &  0.769025 \\
1 &       20pools\_conclusions\_bm\_rm3 &    0.294971 &     0.298600 &  0.698183 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/2031853054.py:38: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_bm_rm3', '21'+ folders[i] + '_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21pools\_conclusionstuned\_bm\_rm3 &    0.486435 &     0.457238 &  0.742910 \\
1 &       21pools\_conclusions\_bm\_rm3 &    0.496191 &     0.444897 &  0.718457 \\
\bottomrule
\end{tabular}



In [6]:
import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels, names=[folders[i] + '_tuned_bm_rm3', folders[i] + '_orignal_bm_rm3'],
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_bm_rm3', '20' + folders[i] + '_bm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_bm_rm3', '21'+ folders[i] + '_bm_rm3'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            pipe_qe.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_bm_rm3.csv")
#             pipe_qe.to_csv(path_or_buf=f"/workspace/src/runs_bm25_rm3/{folders[i]}_tuned_bm_rm3.csv")
            i+=1


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.452738
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c5d9f0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18ff8920 at 0x7f9f182ce930>>) fb_terms=10', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c5d9f0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18ff8920 at 0x7f9f182ce930>>) fb_docs=8', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c5d9f0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x55dc18ff8920 at 0x7f9f182ce930>>) fb_lambda=0.4']
Fold 2
Best ndcg_cut_5 is 0.285510
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f9f19c5d9f0 jcla

/tmp/ipykernel_1615848/3999278728.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics, qrels, names=[folders[i] + '_tuned_bm_rm3', folders[i] + '_orignal_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    marco-pretrained\_tuned\_bm\_rm3 &    0.344765 &     0.314022 &  0.652304 \\
1 &  marco-pretrained\_orignal\_bm\_rm3 &    0.335913 &     0.305915 &  0.620544 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/3999278728.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics20, qrels, names=['20' + folders[i] + 'tuned_bm_rm3', '20' + folders[i] + '_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20marco-pretrainedtuned\_bm\_rm3 &    0.255034 &     0.263154 &  0.673901 \\
1 &       20marco-pretrained\_bm\_rm3 &    0.272867 &     0.259365 &  0.618440 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1615848/3999278728.py:38: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe, bm25_for_qe], topics21, qrels, names=['21' + folders[i] + 'tuned_bm_rm3', '21'+ folders[i] + '_bm_rm3'],


\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21marco-pretrainedtuned\_bm\_rm3 &    0.432702 &     0.363872 &  0.631139 \\
1 &       21marco-pretrained\_bm\_rm3 &    0.397697 &     0.351533 &  0.622606 \\
\bottomrule
\end{tabular}

